In [ ]:
cd /Users/andreibirladeanu/Documents/Data

In [ ]:
import pandas as pd
data = pd.read_excel('rinab_processing.xlsx')


In [ ]:
# creating one series for subject ID's
filtered_subjects = pd.Series(data=data['Subject_id_FILTERED'])

In [ ]:
# remove filtered_subjects column from data so that I can merge it later
data = data.iloc[:,1:] 
#.ix[row slice, column slice] 

In [ ]:
rinab_data = data.merge(filtered_subjects, how='inner', left_on='Subject_ID', right_on='Subject_id_FILTERED')

In [ ]:
#pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
rinab_data = rinab_data.iloc[:,:-1]
rinab_data = rinab_data.sort_values(by='Subject_ID', ignore_index=True)

In [ ]:
rinab_data

In [ ]:
#rinab_data.to_excel("rinab_processed.xlsx")

In [ ]:
# taking total of each row
rinab_data["total"]= rinab_data.iloc[:,1:].sum(axis=1)

In [ ]:
rinab_data

In [ ]:
#rinab_nona = rinab_data.dropna()

In [ ]:
rinab_nona.total.hist()

In [ ]:
rinab_nona.total.describe()

In [ ]:
rinab_nona.total.var()

In [ ]:
#number of bins = sqrt(total no of points)
#bin width = (max-min) / number of bins

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# code from https://github.com/AnalyticsInsightsNinja/PythonVisualAnalytics/blob/master/01_AV_Adavced_Histogram.ipynb
# Plot the Histogram from the random data

def histogram_plot(column, title='Rinab', ylabel='frequency', xlabel='rinab', bin_no=7, legend_title='N=61'):
    fig, ax = plt.subplots(figsize=(8,8))
    counts, bins, patches = ax.hist(column, bins=bin_no, facecolor='mediumaquamarine', edgecolor='gray')

    ax.set_xticks(bins.round(2))
    plt.xticks(rotation=0)

    # Set the graph title and axes titles
    plt.title(title, fontsize=20)
    plt.ylabel(ylabel, fontsize=15)
    plt.xlabel(xlabel, fontsize=15)

    # Calculate bar centre to display the count of data points and %
    bin_x_centers = 0.5 * np.diff(bins) + bins[:-1]
    bin_y_centers = ax.get_yticks()[1] * 0.25

    #Display the the count of data points and % for each bar in histogram
    for i in range(len(bins)-1):
        bin_label = "{0:,}".format(counts[i]) + "  ({0:,.2f}%)".format((counts[i]/counts.sum())*100)
        plt.text(bin_x_centers[i], bin_y_centers, bin_label, rotation=90, rotation_mode='anchor')

    plt.legend(fontsize=10, title=legend_title)
    

In [ ]:
histogram_plot(rinab_nona.total, title='Rinab without NA', legend_title='N=55', xlabel='Rinab score')
histogram_plot(rinab_data.total, title='Rinab with NA', legend_title='N=61', xlabel='Rinab score')



In [ ]:
rinab_data.total.apply(np.log).hist()

In [ ]:
rinab_data.total.apply(np.log)

In [ ]:
# now trying to merge the DAI with the previous dataset
data2 = pd.read_excel('Rinab+DAI+age.xlsx')

In [ ]:
# setting a common index so I can merge them
rinab_data=rinab_data.set_index('Subject_ID')
data2=data2.set_index('Subject_ID')

In [ ]:
# merging the two datasets
rinab_dai_age = data2.merge(rinab_data, how='right', left_on='Subject_ID', right_on='Subject_ID')
rinab_dai_age

In [ ]:
cd ~/Documents/Data

In [ ]:
rinab_dai_age.to_excel('final_rinab+dai.xlsx', na_rep='NA')

In [ ]:
# Scores on Rinab for participants with missing q's B3 and B4
# loc selects index by label, iloc by position
rinab_data.loc[[1059,1195,1250],'total']

In [ ]:
# the mean of the rinab total score excluding those NA subjects rounded to 22 decimals
round(rinab_nona.total.mean(), 2), round(rinab_nona.total.median(), 2) 

In [ ]:
# Scores on Rinab for participants with missing q A3
rinab_data.loc[[1112,1188,1217,1195],'total']

In [ ]:
# compute Spearman correlation (due to data being ordinal and non-normally distributed)
rinab_dai_age.corr(method='spearman')

In [ ]:
from scipy.stats import spearmanr
import pandas as pd

def calculate_pvalues(df):
    df = df._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(spearmanr(df[r], df[c],nan_policy='omit')[1], 4)
    return pvalues

In [ ]:
calculate_pvalues(rinab_dai_age) 

In [ ]:
histogram.plot(rinab_dai_age['DAIscoreT1'], title='DAI histogram', ylabel='frequency', xlabel='DAI score', bin_no=[0,1,2,3,4,5,6,7,8], legend_title='N=55')

In [ ]:
rinab_dai_age.plot.scatter(x='DAIscoreT1', y='total')

In [ ]:
rinab_dai_age.boxplot(column='total', by='DAIscoreT1')